In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta

In [2]:
CNMSshvol_df = pd.read_csv('F/20210212.txt', sep='|', header=0)
CNMSshvol_df['Date'] =  pd.to_datetime(CNMSshvol_df['Date'], errors='coerce', format='%Y%m%d')

In [3]:
CNMSshvol_df

,Date,Symbol,ShortVolume,ShortExemptVolume,TotalVolume,Market
0,2021-02-12,A,98624.0,6.0,304445.0,"B,Q,N"
1,2021-02-12,AA,414786.0,1121.0,1068127.0,"B,Q,N"
2,2021-02-12,AAA,114.0,0.0,116.0,"Q,N"
3,2021-02-12,AAAU,21799.0,0.0,114520.0,"Q,N"
4,2021-02-12,AAC/U,6904.0,0.0,336370.0,"Q,N"
...,...,...,...,...,...,...
9482,2021-02-12,ZWRKU,8714.0,0.0,101032.0,"B,Q,N"
9483,2021-02-12,ZYME,37226.0,0.0,116882.0,"B,Q,N"
9484,2021-02-12,ZYNE,2289583.0,154854.0,7798350.0,"B,Q,N"
9485,2021-02-12,ZYXI,38630.0,1160.0,71296.0,"B,Q,N"


In [13]:
def calculate_change(start, end):
  return round(((end - start)/start)*100, 2)

#this needs to be converted to an array (should be okay because yfticker takes arrays as an arg)
ticker = 'AMC'

open_price = yf.Ticker(ticker).history(period='max')['Open']
close_price = yf.Ticker(ticker).history(period='max')['Close']
df = calculate_change(open_price,close_price)
dataframe=pd.DataFrame(df)
dataframe['Ticker']=ticker

dataframe

,0,Ticker
Date,,
2013-12-18,-1.46,AMC
2013-12-19,3.12,AMC
2013-12-20,0.15,AMC
2013-12-23,-0.25,AMC
2013-12-24,-1.06,AMC
...,...,...
2021-02-08,-10.17,AMC
2021-02-09,-5.34,AMC
2021-02-10,1.58,AMC


In [14]:
today = datetime.today()
#This date is hardcoded but needs to be a variable as well
start_date = today - timedelta(days=15)
filename = start_date.strftime("%Y%m%d")
finra = pd.DataFrame()
bx = pd.DataFrame()
psx = pd.DataFrame()

days_to_scrape = start_date
while (days_to_scrape < today):
    filename = days_to_scrape.strftime("%Y%m%d")
    try:
        file = pd.read_csv('F/'+filename+'.txt', sep='|', header=0)
        finra = pd.concat([finra, file])
        file = pd.read_csv('B/'+filename+'.txt', sep='|', header=0)
        bx = pd.concat([bx, file])
        file = pd.read_csv('P/'+filename+'.txt', sep='|', header=0)
        psx = pd.concat([psx, file])
        
        print('got data for '+ filename)
    except:
        print('no data for ' + filename)


    days_to_scrape += timedelta(days=1)
    

got data for 20210129
no data for 20210130
no data for 20210131
got data for 20210201
got data for 20210202
got data for 20210203
got data for 20210204
got data for 20210205
no data for 20210206
no data for 20210207
got data for 20210208
got data for 20210209
got data for 20210210
got data for 20210211
got data for 20210212


# Calculate shortvol/totalvol for all markets

In [15]:
import warnings
warnings.filterwarnings('ignore')

finra_ticker = finra[finra['Symbol'] == ticker]
bx_ticker = bx[bx['SYMBOL'] == ticker]
psx_ticker = psx[psx['SYMBOL'] == ticker]

for i, row in finra_ticker.iterrows():
    finra_ticker.at[i,'volume_percent'] = row['ShortVolume']/row['TotalVolume']
    
for i, row in bx_ticker.iterrows():
    bx_ticker.at[i,'volume_percent'] =  row['SHORT VOLUME']/row['TOTAL VOLUME']
    
for i, row in psx_ticker.iterrows():
    psx_ticker.at[i,'volume_percent'] =  row['SHORT VOLUME']/row['TOTAL VOLUME']

In [16]:
finra_ticker

,Date,Symbol,ShortVolume,ShortExemptVolume,TotalVolume,Market,volume_percent
377,20210129,AMC,147053814.0,6307292.0,285512338.0,"B,Q,N",0.515052
379,20210201,AMC,76933491.0,986909.0,158404179.0,"B,Q,N",0.533923
385,20210202,AMC,120244517.0,5744009.0,228128789.0,"B,Q,N",0.527090
379,20210203,AMC,61535474.0,3918022.0,115251580.0,"B,Q,N",0.533923
382,20210204,AMC,44880962.0,2248578.0,83053098.0,"B,Q,N",0.535361
382,20210205,AMC,59444755.0,2116603.0,111036783.0,"B,Q,N",0.535361
387,20210208,AMC,39970406.0,1810126.0,76594692.0,"B,Q,N",0.521843
389,20210209,AMC,20727999.0,1536449.0,58293286.0,"B,Q,N",0.334914
392,20210210,AMC,24840204.0,1327153.0,87838231.0,"B,Q,N",0.282795
388,20210211,AMC,11643237.0,123722.0,28600493.0,"B,Q,N",0.407099


In [17]:
bx_ticker

,DATE,SYMBOL,SHORT VOLUME,TOTAL VOLUME,MARKET,volume_percent
265,20210129,AMC,1915135,2258917,B,0.847811
258,20210201,AMC,890528,1193465,B,0.746170
261,20210202,AMC,1650631,2285057,B,0.722359
250,20210203,AMC,537903,656694,B,0.819108
267,20210204,AMC,913884,1230441,B,0.742729
273,20210205,AMC,502642,671293,B,0.856284
273,20210208,AMC,518985,606090,B,0.856284
269,20210209,AMC,315696,402900,B,0.783559
284,20210210,AMC,240512,321817,B,0.747356
277,20210211,AMC,76935,103716,B,0.741785


In [18]:
psx_ticker

,DATE,SYMBOL,SHORT VOLUME,TOTAL VOLUME,MARKET,volume_percent
227,20210129,AMC,168839,279862,X,0.655289
215,20210201,AMC,404707,591350,X,0.684378
218,20210202,AMC,357945,530225,X,0.773101
221,20210203,AMC,219004,321738,X,0.680690
218,20210204,AMC,321038,415260,X,0.773101
227,20210205,AMC,98946,150996,X,0.655289
226,20210208,AMC,162361,184596,X,0.760979
233,20210209,AMC,71603,141797,X,0.504968
251,20210210,AMC,85120,284927,X,0.298743
232,20210211,AMC,112040,148052,X,0.756761


In [19]:
psx_ticker=psx_ticker.rename(columns={"SYMBOL": "symbol", 'DATE':'date'})
bx_ticker=bx_ticker.rename(columns={"SYMBOL": "symbol",'DATE':'date'})
finra_ticker=finra_ticker.rename(columns={"Symbol": "symbol",'Date':'date'})

In [20]:
final_df =  pd.merge(pd.merge(psx_ticker, bx_ticker, on=['symbol','date']),finra_ticker, on=['symbol','date'])

In [21]:
final_df

,date,symbol,SHORT VOLUME_x,TOTAL VOLUME_x,MARKET_x,volume_percent_x,SHORT VOLUME_y,TOTAL VOLUME_y,MARKET_y,volume_percent_y,ShortVolume,ShortExemptVolume,TotalVolume,Market,volume_percent
0,20210129,AMC,168839,279862,X,0.655289,1915135,2258917,B,0.847811,147053814.0,6307292.0,285512338.0,"B,Q,N",0.515052
1,20210201,AMC,404707,591350,X,0.684378,890528,1193465,B,0.746170,76933491.0,986909.0,158404179.0,"B,Q,N",0.533923
2,20210202,AMC,357945,530225,X,0.773101,1650631,2285057,B,0.722359,120244517.0,5744009.0,228128789.0,"B,Q,N",0.527090
3,20210203,AMC,219004,321738,X,0.680690,537903,656694,B,0.819108,61535474.0,3918022.0,115251580.0,"B,Q,N",0.533923
4,20210204,AMC,321038,415260,X,0.773101,913884,1230441,B,0.742729,44880962.0,2248578.0,83053098.0,"B,Q,N",0.535361
5,20210205,AMC,98946,150996,X,0.655289,502642,671293,B,0.856284,59444755.0,2116603.0,111036783.0,"B,Q,N",0.535361
6,20210208,AMC,162361,184596,X,0.760979,518985,606090,B,0.856284,39970406.0,1810126.0,76594692.0,"B,Q,N",0.521843
7,20210209,AMC,71603,141797,X,0.504968,315696,402900,B,0.783559,20727999.0,1536449.0,58293286.0,"B,Q,N",0.334914
8,20210210,AMC,85120,284927,X,0.298743,240512,321817,B,0.747356,24840204.0,1327153.0,87838231.0,"B,Q,N",0.282795
9,20210211,AMC,112040,148052,X,0.756761,76935,103716,B,0.741785,11643237.0,123722.0,28600493.0,"B,Q,N",0.407099
